# This is the Preprocessing codes to analyze the running invervention dataset

## Gitrepo
    https://github.com/Vincent-wq/interventionMDD
    
## Dataset:
    0. Data source: [https://openneuro.org/datasets/ds003799/versions/1.0.0]( Openneuro)
    1. Imaging data MRI *3 sessions;
    2. Depression measures: Description: German version of the Center for Epidemiological Studies Depression Scale(CES-D). Hautzinger, M., Bailer, M., Hofmeister, D., & Keller, F. (2012). Allgemeine Depressionsskala (ADS). Manual (2. Auflage). Göttingen: Hogrefe Verlag GmbH. This self-assessing scale comprises 20 items covering depressive symptoms in the emotional, motivational, cognitive, somatic, and motoric domain. Participants had to indicate the extent to which the given conditions apply to them in the last week on a four-point scale ranging from “infrequent” to “the most time”.

## Preprocessing steps:
    1. Pre-registration: https://osf.io/yg5bu/
    2. Preprocessing steps: 
        2.1 fMRIPrep 20.2.7
        2.2 QA
        2.3 TBD


In [1]:
# libs and envs
import sys
proj_path_str='/scratch/interventionMDD'
sys.path.append(proj_path_str)
from pathlib import Path
import pandas as pd
import numpy as np

# main PATH
main_dir = Path(proj_path_str)
data_dir = main_dir / 'data' 
fig_dir =  main_dir / 'figs' 

# read tabular files 
participant_file = data_dir / 'participants.tsv'  # Inormation from download database.
mdd_score_file   = data_dir / 'phenotype_CES-D.tsv'  # Inormation from download database.

# read and select UPDRS-3data
sub_data = pd.read_csv(participant_file, sep='\t')
sub_data.index=sub_data['participant_id']
sub_data=sub_data.drop(columns=['participant_id']).copy()
sub_col_list = ['participant_id', 'group', 'age', 'sex', 'size', 'weight'];

# read and select MOCA data
mdd_data  = pd.read_csv(mdd_score_file, sep='\t')
mdd_data.index = mdd_data['participant_id']
mdd_data=mdd_data.drop(columns=['participant_id']).copy()
moca_list = ['participant_id', 'CES-D_1', 'CES-D_2', 'CES-D_3'];

##join data
data_df = sub_data.join(mdd_data, on = ['participant_id'], how='left').copy()

In [ ]:
# Freesurfer brain measures
fs_ses1_dir  =  main_dir / "scripts" / "collect_fs_measures" / "ses-1" 
fs_ses2_dir  =  main_dir / "scripts" / "collect_fs_measures" / "ses-2" 
fs_ses3_dir  =  main_dir / "scripts" / "collect_fs_measures" / "ses-3" 

files_2_read={'seg'      : ['aseg_stats.txt', 'wmparc_stats.txt'],
              'Destrieux': {'ct': '.a2009s.thickness.txt',  'area':'.a2009s.area.txt',   'volume':'.a2009s.volume.txt'},
              'DKT'      : {'ct': '.DKTatlas.thickness.txt','area':'.DKTatlas.area.txt', 'volume':'.DKTatlas.volume.txt'}}

for i_group in range(n_groups):
    """
    Loop to gather all the freesurfer outputs for ET/PD/NC MNI group.
    Output: all_data (dataframe).
    """
    print('Reading ' , GROUPS[i_group], ' freesurfer stats data...')
    raw_data_path = FS_DIR/GROUPS[i_group]
    
    # preparing files
    subcortical_file = raw_data_path / (files_2_read['seg'][0]); wm_file = raw_data_path / (files_2_read['seg'][1]);
    # Des parcellation
    lh_Des_ct_file = raw_data_path / ('lh'+files_2_read['Destrieux']['ct']);  rh_Des_ct_file = raw_data_path /  ('rh'+files_2_read['Destrieux']['ct']);
    lh_Des_vol_file = raw_data_path / ('lh'+files_2_read['Destrieux']['volume']); rh_Des_vol_file = raw_data_path / ('rh'+files_2_read['Destrieux']['volume']);
    lh_Des_area_file = raw_data_path / ('lh'+files_2_read['Destrieux']['area']); rh_Des_area_file = raw_data_path / ('rh'+files_2_read['Destrieux']['area']);
    lh_Des_meancurv_file = raw_data_path / ('lh'+files_2_read['Destrieux']['meancurv']); rh_Des_meancurv_file = raw_data_path / ('rh'+files_2_read['Destrieux']['meancurv']);
    # DKT parcellation
    lh_DKT_area_file = raw_data_path / ('lh'+files_2_read['DKT']['area']);       rh_DKT_area_file = raw_data_path / ('rh'+files_2_read['DKT']['area']);
    lh_DKT_ct_file = raw_data_path / ('lh'+files_2_read['DKT']['ct']);        rh_DKT_ct_file = raw_data_path /  ('rh'+files_2_read['DKT']['ct']);
    lh_DKT_vol_file = raw_data_path / ('lh'+files_2_read['DKT']['volume']);       rh_DKT_vol_file = raw_data_path / ('rh'+files_2_read['DKT']['volume']);
    lh_DKT_meancurv_file = raw_data_path / ('lh'+files_2_read['DKT']['meancurv']);       rh_DKT_meancurv_file = raw_data_path / ('rh'+files_2_read['DKT']['meancurv']);
    ## drop_list
    aseg_drop = ["EstimatedTotalIntraCranialVol"]; 
    wm_drop = ["MaskVol", "EstimatedTotalIntraCranialVol", "CerebralWhiteMatterVol", "rhCerebralWhiteMatterVol", "lhCerebralWhiteMatterVol"];
    parc_drop = ["BrainSegVolNotVent", "eTIV"]; 
    sub_list[i_group].loc[:, 'site']='mni_001'
    sub_list[i_group].loc[:, 'group'] = sub_list[i_group].loc[:, 'diagnosis'];
    # read files
    subcortical_tab = pd.read_csv(subcortical_file, sep='\t', header=0, index_col=0); 
    subcortical_tab['eTIV']=subcortical_tab['EstimatedTotalIntraCranialVol']
    subcortical_tab.drop(aseg_drop, axis=1, inplace=True);
    # site
    res = sub_list[i_group].join(subcortical_tab, how='left');
    wm_tab = pd.read_csv(wm_file, sep='\t', header=0, index_col=0); wm_tab.drop(wm_drop, axis=1, inplace=True);
    res1   = res.join(wm_tab, how='left');
    # read Des/DKT parcelation data
    lh_Des_ct_tab  = pd.read_csv(lh_Des_ct_file,  sep='\t', header=0, index_col=0);       lh_Des_ct_tab.drop(parc_drop, axis=1, inplace=True);
    rh_Des_ct_tab  = pd.read_csv(rh_Des_ct_file,  sep='\t', header=0, index_col=0);       rh_Des_ct_tab.drop(parc_drop, axis=1, inplace=True);
    lh_Des_vol_tab = pd.read_csv(lh_Des_vol_file, sep='\t', header=0, index_col=0);       lh_Des_vol_tab.drop(parc_drop, axis=1, inplace=True);
    rh_Des_vol_tab = pd.read_csv(rh_Des_vol_file, sep='\t', header=0, index_col=0);       rh_Des_vol_tab.drop(parc_drop, axis=1, inplace=True);
    lh_Des_area_tab = pd.read_csv(lh_Des_area_file, sep='\t', header=0, index_col=0);     lh_Des_area_tab.drop(parc_drop, axis=1, inplace=True);
    rh_Des_area_tab = pd.read_csv(rh_Des_area_file, sep='\t', header=0, index_col=0);     rh_Des_area_tab.drop(parc_drop, axis=1, inplace=True);
    lh_Des_curv_tab = pd.read_csv(lh_Des_meancurv_file, sep='\t', header=0, index_col=0); lh_Des_curv_tab.drop(parc_drop, axis=1, inplace=True);
    rh_Des_curv_tab = pd.read_csv(rh_Des_meancurv_file, sep='\t', header=0, index_col=0); rh_Des_curv_tab.drop(parc_drop, axis=1, inplace=True);
    # DKT atlas
    lh_DKT_ct_tab  = pd.read_csv(lh_DKT_ct_file,  sep='\t', header=0, index_col=0);       lh_DKT_ct_tab.drop(parc_drop, axis=1, inplace=True);
    rh_DKT_ct_tab  = pd.read_csv(rh_DKT_ct_file,  sep='\t', header=0, index_col=0);       rh_DKT_ct_tab.drop(parc_drop, axis=1, inplace=True);
    lh_DKT_vol_tab = pd.read_csv(lh_DKT_vol_file, sep='\t', header=0, index_col=0);       lh_DKT_vol_tab.drop(parc_drop, axis=1, inplace=True);
    rh_DKT_vol_tab = pd.read_csv(rh_DKT_vol_file, sep='\t', header=0, index_col=0);       rh_DKT_vol_tab.drop(parc_drop, axis=1, inplace=True);
    lh_DKT_area_tab = pd.read_csv(lh_DKT_area_file, sep='\t', header=0, index_col=0);     lh_DKT_area_tab.drop(parc_drop, axis=1, inplace=True);
    rh_DKT_area_tab = pd.read_csv(rh_DKT_area_file, sep='\t', header=0, index_col=0);     rh_DKT_area_tab.drop(parc_drop, axis=1, inplace=True);
    lh_DKT_curv_tab = pd.read_csv(lh_DKT_meancurv_file, sep='\t', header=0, index_col=0); lh_DKT_curv_tab.drop(parc_drop, axis=1, inplace=True);
    rh_DKT_curv_tab = pd.read_csv(rh_DKT_meancurv_file, sep='\t', header=0, index_col=0); rh_DKT_curv_tab.drop(parc_drop, axis=1, inplace=True);
    # merge Des/DKT parcelation data
    seg_Des_tab=       res1.join(lh_Des_ct_tab, how='left');  seg_Des_tab=seg_Des_tab.join(rh_Des_ct_tab, how='left'); 
    seg_Des_tab=seg_Des_tab.join(lh_Des_vol_tab,how='left');  seg_Des_tab=seg_Des_tab.join(rh_Des_vol_tab,how='left'); 
    seg_Des_tab=seg_Des_tab.join(lh_Des_area_tab,how='left'); seg_Des_tab=seg_Des_tab.join(rh_Des_area_tab,how='left');
    seg_Des_tab=seg_Des_tab.join(lh_Des_curv_tab,how='left'); seg_Des_tab=seg_Des_tab.join(rh_Des_curv_tab,how='left');
    
    seg_DKT_tab=res1.join(lh_DKT_ct_tab, how='left');         seg_DKT_tab=seg_DKT_tab.join(rh_DKT_ct_tab, how='left'); 
    seg_DKT_tab=seg_DKT_tab.join(lh_DKT_vol_tab,how='left');  seg_DKT_tab=seg_DKT_tab.join(rh_DKT_vol_tab,how='left'); 
    seg_DKT_tab=seg_DKT_tab.join(lh_DKT_area_tab,how='left'); seg_DKT_tab=seg_DKT_tab.join(rh_DKT_area_tab,how='left'); 
    seg_DKT_tab=seg_DKT_tab.join(lh_DKT_curv_tab,how='left'); seg_DKT_tab=seg_DKT_tab.join(rh_DKT_curv_tab,how='left'); 
    # return data
    group_data[GROUPS[i_group]]={'Des': seg_Des_tab, 'DKT':seg_DKT_tab}
all_data = {'Des': pd.concat([group_data['ET']['Des'], group_data['NC']['Des']]), 'DKT': pd.concat([group_data['ET']['DKT'], group_data['NC']['DKT']])}
for k, v in all_data.items():
    v.index   = [x.replace('-','_') for x in v.index]
    v.columns = [x.replace('-','_') for x in v.columns]
for x in GROUPS:
    print(x , len(group_data[x]['Des']))

## Generate subject list for QA

In [ ]:
# Generate files for QA
# output QA subject list 
mdd_qa_file   = main_dir / "scripts" / "QA" / 'runningMDD_qa_list.csv'  # Information from download database.
data_df.to_csv(mdd_qa_file, columns=[], header=False)
#generate the subject list for freesurfer results collection
print("Subject list space seperated: ")
print(' '.join((data_df.index)))